In [1]:
import numpy as np
from sklearn import preprocessing

In [2]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')

In [3]:
unscaled_inputs_all = raw_csv_data[:,1:-1]

In [4]:
targets_all = raw_csv_data[:,-1]

In [5]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)
unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

In [6]:
num_one_targets = int(np.sum(targets_all))

In [7]:
zero_targets_counter = 0

In [8]:
indices_to_remove = []

In [9]:
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

In [10]:
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [11]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [12]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [13]:
samples_count = shuffled_inputs.shape[0]

In [14]:
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

In [15]:
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

In [16]:
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

In [17]:
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [18]:
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1768.0 3579 0.49399273540095
238.0 447 0.5324384787472036
231.0 448 0.515625


In [19]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [20]:
import numpy as np
import tensorflow as tf

In [21]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

In [22]:
npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

In [23]:
npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype('float'), npz['targets'].astype(int)

In [24]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2)

model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          callbacks = [early_stopping],
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Epoch 1/100
36/36 - 0s - 9ms/step - accuracy: 0.6306 - loss: 0.6130 - val_accuracy: 0.7494 - val_loss: 0.5290
Epoch 2/100
36/36 - 0s - 697us/step - accuracy: 0.7740 - loss: 0.4757 - val_accuracy: 0.8009 - val_loss: 0.4551
Epoch 3/100
36/36 - 0s - 672us/step - accuracy: 0.7935 - loss: 0.4213 - val_accuracy: 0.8009 - val_loss: 0.4190
Epoch 4/100
36/36 - 0s - 642us/step - accuracy: 0.7988 - loss: 0.3975 - val_accuracy: 0.8076 - val_loss: 0.3972
Epoch 5/100
36/36 - 0s - 640us/step - accuracy: 0.7997 - loss: 0.3843 - val_accuracy: 0.8054 - val_loss: 0.3863
Epoch 6/100
36/36 - 0s - 637us/step - accuracy: 0.8027 - loss: 0.3764 - val_accuracy: 0.8054 - val_loss: 0.3876
Epoch 7/100
36/36 - 0s - 639us/step - accuracy: 0.8094 - loss: 0.3706 - val_accuracy: 0.8031 - val_loss: 0.3892


In [26]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step - accuracy: 0.7740 - loss: 0.4108
Test loss: 0.40. Test accuracy: 79.46%
